In [ ]:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lnRR_allcause = np.log(1.016)
lnRR_sigma_allcause = (np.log(1.021) - np.log(1.016))/1.96

lnRR_ncd = np.log(1.016)
lnRR_sigma_ncd = (np.log(1.021) - np.log(1.016))/1.96

lnRR_resp = np.log(1.042)
lnRR_sigma_resp = (np.log(1.055) - np.log(1.042))/1.96

lnRR_copd = np.log(1.060)
lnRR_sigma_copd = (np.log(1.080) - np.log(1.060))/1.96

lnRR_cvd = np.log(1.017)
lnRR_sigma_cvd = (np.log(1.024) - np.log(1.017))/1.96

lnRR_ihd = np.log(1.024)
lnRR_sigma_ihd = (np.log(1.040) - np.log(1.024))/1.96

LCC = 44.5 #umbral para el ozono troposférico (O₃) de 8 horas (MDA8h).

In [ ]:
data = pd.read_csv('ozone_17_mort_est_total(1119).csv')

In [ ]:
start = time.process_time()
results = []

for t in range(0, 61607, 1):

    ## Extraer población y mortalidad
    pop_all_age       = 7873316
    pop_all_age_sigma = pop_all_age * 0.05
    pop_25_age        = 5268211
    pop_25_age_sigma  = pop_25_age * 0.05

    allcause_25_mort       = 504810
    allcause_25_mort_sigma = allcause_25_mort* 0.05
    resp_25_mort           = 14926
    resp_25_mort_sigma     = resp_25_mort* 0.05
    copd_25_mort           = 8211
    copd_25_mort_sigma     = copd_25_mort* 0.05
    cvd_25_mort            = 4563
    cvd_25_mort_sigma      = cvd_25_mort* 0.05
    ihd_25_mort            = 44646
    ihd_25_mort_sigma      = ihd_25_mort * 0.05
    ncd_25_mort            = 171525
    ncd_25_mort_sigma      = ncd_25_mort * 0.05

    o3_mda8h_urban  = 15.8
    o3_mda8h_urban_dose = max(0, o3_mda8h_urban - LCC)

    ## Simulación Monte Carlo
    pop_all_age_dist      = np.random.normal(pop_all_age, pop_all_age_sigma, 1000)
    pop_25_age_dist       = np.random.normal(pop_25_age , pop_25_age_sigma , 1000)

    allcause_25_mort_dist = np.random.normal(allcause_25_mort, allcause_25_mort_sigma, 1000)
    resp_25_mort_dist     = np.random.normal(resp_25_mort    , resp_25_mort_sigma    , 1000)
    copd_25_mort_dist     = np.random.normal(copd_25_mort    , copd_25_mort_sigma    , 1000)
    cvd_25_mort_dist      = np.random.normal(cvd_25_mort     , cvd_25_mort_sigma     , 1000)
    ihd_25_mort_dist      = np.random.normal(ihd_25_mort     , ihd_25_mort_sigma     , 1000)
    ncd_25_mort_dist      = np.random.normal(ncd_25_mort     , ncd_25_mort_sigma     , 1000)

    RR_dist_allcause = np.random.normal(lnRR_allcause, lnRR_sigma_allcause, 1000)
    RR_dist_resp     = np.random.normal(lnRR_resp    , lnRR_sigma_resp    , 1000)
    RR_dist_copd     = np.random.normal(lnRR_copd    , lnRR_sigma_copd    , 1000)
    RR_dist_cvd      = np.random.normal(lnRR_cvd     , lnRR_sigma_cvd     , 1000)
    RR_dist_ihd      = np.random.normal(lnRR_ihd     , lnRR_sigma_ihd     , 1000)
    RR_dist_ncd      = np.random.normal(lnRR_ncd     , lnRR_sigma_ncd     , 1000)

    ## Cálculo de mortalidades atribuibles
    delta_mort_allcause_urban = allcause_25_mort_dist * (1 - np.exp(- RR_dist_allcause * o3_mda8h_urban_dose / 10))  * pop_25_age_dist / pop_all_age_dist / 100000
    delta_mort_resp_urban     = resp_25_mort_dist     * (1 - np.exp(- RR_dist_resp     * o3_mda8h_urban_dose / 10)) * pop_25_age_dist / pop_all_age_dist / 100000
    delta_mort_copd_urban     = copd_25_mort_dist     * (1 - np.exp(- RR_dist_copd     * o3_mda8h_urban_dose / 10))  * pop_25_age_dist / pop_all_age_dist / 100000
    delta_mort_cvd_urban      = cvd_25_mort_dist      * (1 - np.exp(- RR_dist_cvd      * o3_mda8h_urban_dose / 10))  * pop_25_age_dist / pop_all_age_dist / 100000
    delta_mort_ihd_urban      = ihd_25_mort_dist      * (1 - np.exp(- RR_dist_ihd      * o3_mda8h_urban_dose / 10))  * pop_25_age_dist / pop_all_age_dist / 100000
    delta_mort_ncd_urban      = ncd_25_mort_dist      * (1 - np.exp(- RR_dist_ncd      * o3_mda8h_urban_dose / 10))  * pop_25_age_dist / pop_all_age_dist / 100000

   # Guardamos estadísticos resumen por iteración
    results.append({
        'row_index': t,
        'allcause_median': np.median(delta_mort_allcause_urban),
        'resp_median': np.median(delta_mort_resp_urban),
        'copd_median': np.median(delta_mort_copd_urban),
        'cvd_median': np.median(delta_mort_cvd_urban),
        'ihd_median': np.median(delta_mort_ihd_urban),
        'ncd_median': np.median(delta_mort_ncd_urban),
    })

    if t % 1000 == 0:
        print(f"Iteración {t}")

# Guardar en DataFrame y CSV
results_df = pd.DataFrame(results)
results_df.to_csv('resultados_mortalidad_urbana.csv', index=False)

elapsed = time.process_time() - start
print('Tiempo total:', elapsed, "s")
print("Resultados guardados en 'resultados_mortalidad_urbana.csv'")

Iteración 0
Iteración 1000
Iteración 2000
Iteración 3000
Iteración 4000
Iteración 5000
Iteración 6000
Iteración 7000
Iteración 8000
Iteración 9000
Iteración 10000
Iteración 11000
Iteración 12000
Iteración 13000
Iteración 14000
Iteración 15000
Iteración 16000
Iteración 17000
Iteración 18000
Iteración 19000
Iteración 20000
Iteración 21000
Iteración 22000
Iteración 23000
Iteración 24000
Iteración 25000
Iteración 26000
Iteración 27000
Iteración 28000
Iteración 29000
Iteración 30000
Iteración 31000
Iteración 32000
Iteración 33000
Iteración 34000
Iteración 35000
Iteración 36000
Iteración 37000
Iteración 38000
Iteración 39000
Iteración 40000
Iteración 41000
Iteración 42000
Iteración 43000
Iteración 44000
Iteración 45000
Iteración 46000
Iteración 47000
Iteración 48000
Iteración 49000
Iteración 50000
Iteración 51000
Iteración 52000
Iteración 53000
Iteración 54000
Iteración 55000
Iteración 56000
Iteración 57000
Iteración 58000
Iteración 59000
Iteración 60000
Iteración 61000
Tiempo total: 57.5520